# 🦆 TinySwallow-1.5B DPO Training on Google Colab

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/yourusername/AD_Tech_SLM/blob/main/notebooks/colab_dpo_training_v2.ipynb)

**Direct Preference Optimization (DPO) Training for Japanese Language Model**

- **Model**: SakanaAI/TinySwallow-1.5B-Instruct
- **Target**: 広告技術分野の専門知識向上
- **Framework**: TRL 0.18.1 + Transformers
- **Hardware**: Google Colab GPU (T4/V100/A100)

---

## 🛠️ 1. 環境セットアップ

### GPU確認と必要ライブラリのインストール

In [ ]:
# GPU確認
!nvidia-smi

import torch
print(f"🔧 PyTorch Version: {torch.__version__}")
print(f"🚀 CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"📱 GPU Device: {torch.cuda.get_device_name()}")
    print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

In [ ]:
# 最新の依存関係をインストール
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q transformers==4.52.4
!pip install -q trl==0.18.1
!pip install -q datasets==3.1.0
!pip install -q peft==0.15.0
!pip install -q bitsandbytes==0.44.1
!pip install -q accelerate==1.2.1
!pip install -q wandb
!pip install -q tensorboard

# Colab特有の問題解決
!pip install -q tf-keras

print("✅ 依存関係のインストール完了")

### 必要ライブラリのインポート

In [ ]:
import os
import json
import torch
import logging
import warnings
from datetime import datetime
from typing import Dict, List, Optional

# Transformers & TRL
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)
from trl import DPOTrainer, DPOConfig
from datasets import Dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# Colab用設定
warnings.filterwarnings('ignore')
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("📦 ライブラリインポート完了")
print(f"🔧 TRL Version: {trl.__version__ if 'trl' in globals() else 'Not found'}")
print(f"🤖 Transformers Version: {transformers.__version__ if 'transformers' in globals() else 'Not found'}")

## ⚙️ 2. 設定とデータセット準備

In [ ]:
# トレーニング設定
CONFIG = {
    # モデル設定
    "model_name": "SakanaAI/TinySwallow-1.5B-Instruct",
    "backup_models": [
        "tokyotech-llm/Swallow-1.5b-instruct-hf",
        "rinna/japanese-gpt-neox-3.6b-instruction-sft"
    ],
    
    # トレーニング設定（Colab GPU最適化）
    "num_train_epochs": 2,
    "max_steps": 500,
    "per_device_train_batch_size": 2,
    "per_device_eval_batch_size": 2,
    "gradient_accumulation_steps": 4,
    "learning_rate": 1e-6,
    "warmup_steps": 50,
    "eval_steps": 50,
    "save_steps": 100,
    "logging_steps": 10,
    
    # DPO設定
    "beta": 0.1,
    "max_length": 1024,
    "max_prompt_length": 512,
    
    # LoRA設定
    "lora_r": 16,
    "lora_alpha": 32,
    "lora_dropout": 0.05,
    "target_modules": ["q_proj", "v_proj", "k_proj", "o_proj"],
    
    # データセット設定
    "test_size": 0.1,
    "seed": 42,
    
    # 出力設定
    "output_dir": "./outputs/colab_dpo_training",
}

print("✅ 設定完了")
print(f"📋 使用モデル: {CONFIG['model_name']}")
print(f"🎯 最大ステップ数: {CONFIG['max_steps']}")
print(f"📊 バッチサイズ: {CONFIG['per_device_train_batch_size']}")

### サンプルデータセットの作成

広告技術分野に特化したDPOデータセットを作成します。

In [ ]:
# 広告技術分野のサンプルDPOデータセット
sample_dpo_data = [
    {
        "prompt": "プログラマティック広告のRTBについて説明してください。",
        "chosen": "RTB（Real-Time Bidding）は、広告枠の売買をリアルタイムのオークション形式で行う仕組みです。ユーザーがWebページにアクセスした瞬間に、そのユーザーの属性や閲覧履歴に基づいて、広告主が自動的に入札を行います。最も高い金額を提示した広告主の広告が表示される仕組みで、効率的なターゲティングと費用対効果の向上を実現します。",
        "rejected": "RTBは広告を表示するシステムです。"
    },
    {
        "prompt": "DSPとSSPの違いを教えてください。",
        "chosen": "DSP（Demand-Side Platform）は広告主側のプラットフォームで、広告枠の購入を自動化し、ターゲティングや入札戦略の最適化を行います。一方、SSP（Supply-Side Platform）はメディア側のプラットフォームで、広告枠の販売を自動化し、収益の最大化を図ります。DSPは買い手、SSPは売り手の立場でプログラマティック広告の取引を支援します。",
        "rejected": "DSPとSSPは両方とも広告関連のシステムです。"
    },
    {
        "prompt": "クッキーレス時代の広告ターゲティング手法について説明してください。",
        "chosen": "クッキーレス時代では、以下の手法が重要となります：1) ファーストパーティデータの活用（自社の顧客データベース）、2) コンテキスト広告（Webページの内容に基づいたターゲティング）、3) コホート分析（類似ユーザーグループでの分析）、4) プライバシーサンドボックス技術（Googleが提案する新技術群）、5) IDソリューション（メールアドレスベースの識別子）などがあります。",
        "rejected": "クッキーが使えなくなるので、新しい方法を考える必要があります。"
    },
    {
        "prompt": "アトリビューション分析の重要性について教えてください。",
        "chosen": "アトリビューション分析は、コンバージョンに至るまでの各タッチポイントの貢献度を測定する分析手法です。ユーザーの購買行動は複雑で、複数の広告接触を経てコンバージョンに至るため、ラストクリック以外の接触点の価値も適切に評価する必要があります。これにより、マーケティング予算の最適配分、チャネル間の相互作用の理解、ROIの正確な測定が可能になります。",
        "rejected": "アトリビューション分析は広告の効果を測定する方法です。"
    },
    {
        "prompt": "ヘッダービディングとは何ですか？",
        "chosen": "ヘッダービディング（Header Bidding）は、複数のSSPやアドエクスチェンジが同時に広告枠に入札できる仕組みです。従来のウォーターフォール方式とは異なり、すべての需要源が同じタイミングで競合できるため、より公平で透明性の高いオークションが実現されます。結果として、パブリッシャーの収益向上と広告主の効率的な配信が可能になります。",
        "rejected": "ヘッダービディングは広告枠を売る方法の一つです。"
    }
]

# より多くのサンプルデータを生成（実際のプロジェクトでは実データを使用）
extended_data = []
for i in range(50):  # 50回繰り返してデータを拡張
    for base_data in sample_dpo_data:
        extended_data.append({
            "prompt": base_data["prompt"],
            "chosen": base_data["chosen"],
            "rejected": base_data["rejected"]
        })

print(f"📊 サンプルデータセット作成完了: {len(extended_data)} サンプル")
print(f"📝 サンプル例:")
print(f"  Prompt: {sample_dpo_data[0]['prompt'][:50]}...")
print(f"  Chosen: {sample_dpo_data[0]['chosen'][:50]}...")

## 🤖 3. モデルとトークナイザーの読み込み

In [ ]:
# トークナイザーの読み込み
def load_tokenizer(model_name):
    try:
        tokenizer = AutoTokenizer.from_pretrained(
            model_name,
            trust_remote_code=True
        )
        
        # パディングトークンの設定
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token
            logger.info("🔧 パディングトークンを設定")
        
        return tokenizer
    except Exception as e:
        logger.error(f"❌ トークナイザー読み込みエラー: {e}")
        return None

# メインモデルを試行
tokenizer = load_tokenizer(CONFIG["model_name"])

# 失敗した場合はバックアップモデルを試行
if tokenizer is None:
    for backup_model in CONFIG["backup_models"]:
        logger.info(f"🔄 バックアップモデルを試行: {backup_model}")
        tokenizer = load_tokenizer(backup_model)
        if tokenizer is not None:
            CONFIG["model_name"] = backup_model
            break

if tokenizer is None:
    raise ValueError("❌ すべてのモデルの読み込みに失敗しました")

print(f"✅ トークナイザー読み込み完了: {CONFIG['model_name']}")
print(f"📏 語彙サイズ: {len(tokenizer)}")

In [ ]:
# 量子化設定（GPU メモリ節約）
def create_bnb_config():
    return BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

# モデルの読み込み
def load_model(model_name, use_quantization=True):
    try:
        model_kwargs = {
            "trust_remote_code": True,
            "torch_dtype": torch.bfloat16,
            "device_map": "auto",
        }
        
        if use_quantization:
            model_kwargs["quantization_config"] = create_bnb_config()
            logger.info("🔧 4bit量子化を使用")
        
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            **model_kwargs
        )
        
        return model
    except Exception as e:
        logger.error(f"❌ モデル読み込みエラー: {e}")
        return None

# モデル読み込み実行
model = load_model(CONFIG["model_name"])

if model is None:
    raise ValueError("❌ モデルの読み込みに失敗しました")

print(f"✅ モデル読み込み完了: {CONFIG['model_name']}")
print(f"📊 パラメータ数: {sum(p.numel() for p in model.parameters()):,}")
print(f"💾 モデルサイズ: {sum(p.numel() * p.element_size() for p in model.parameters()) / 1024**3:.2f} GB")

### LoRA設定の適用

In [ ]:
# LoRA設定
lora_config = LoraConfig(
    r=CONFIG["lora_r"],
    lora_alpha=CONFIG["lora_alpha"],
    target_modules=CONFIG["target_modules"],
    lora_dropout=CONFIG["lora_dropout"],
    bias="none",
    task_type="CAUSAL_LM",
)

# 量子化されたモデルの準備
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

# 訓練可能パラメータの確認
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())
trainable_percentage = 100 * trainable_params / total_params

print(f"🔧 LoRA設定適用完了")
print(f"📈 訓練可能パラメータ: {trainable_params:,} ({trainable_percentage:.2f}%)")
print(f"📊 総パラメータ数: {total_params:,}")

# メモリ使用量確認
if torch.cuda.is_available():
    print(f"💾 GPU メモリ使用量: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")

## 📊 4. データセット準備

In [ ]:
# データセットの作成と分割
dataset = Dataset.from_list(extended_data)

# 訓練・検証データの分割
split_dataset = dataset.train_test_split(
    test_size=CONFIG["test_size"],
    seed=CONFIG["seed"]
)

train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

print(f"✅ データセット準備完了")
print(f"📊 訓練データ: {len(train_dataset)} サンプル")
print(f"📊 検証データ: {len(eval_dataset)} サンプル")

# サンプルデータの確認
print(f"\n📝 訓練データのサンプル:")
sample = train_dataset[0]
print(f"  Prompt: {sample['prompt'][:100]}...")
print(f"  Chosen: {sample['chosen'][:100]}...")
print(f"  Rejected: {sample['rejected'][:50]}...")

## 🚀 5. DPOトレーニング設定

In [ ]:
# 出力ディレクトリの作成
os.makedirs(CONFIG["output_dir"], exist_ok=True)

# DPOConfig設定（TRL 0.18.1対応）
training_args = DPOConfig(
    output_dir=CONFIG["output_dir"],
    num_train_epochs=CONFIG["num_train_epochs"],
    max_steps=CONFIG["max_steps"],
    per_device_train_batch_size=CONFIG["per_device_train_batch_size"],
    per_device_eval_batch_size=CONFIG["per_device_eval_batch_size"],
    gradient_accumulation_steps=CONFIG["gradient_accumulation_steps"],
    learning_rate=CONFIG["learning_rate"],
    warmup_steps=CONFIG["warmup_steps"],
    eval_steps=CONFIG["eval_steps"],
    save_steps=CONFIG["save_steps"],
    logging_steps=CONFIG["logging_steps"],
    eval_strategy="steps",
    save_strategy="steps",
    save_total_limit=3,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    fp16=True,  # GPU用
    remove_unused_columns=False,
    report_to=["tensorboard"],  # Colab用
    # DPO固有のパラメータ
    beta=CONFIG["beta"],
    max_length=CONFIG["max_length"],
    max_prompt_length=CONFIG["max_prompt_length"],
)

print(f"✅ DPOConfig設定完了")
print(f"📋 最大ステップ数: {CONFIG['max_steps']}")
print(f"🎯 学習率: {CONFIG['learning_rate']}")
print(f"🔥 DPO Beta: {CONFIG['beta']}")

In [ ]:
# DPOTrainer の初期化（TRL 0.18.1対応）
trainer = DPOTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    processing_class=tokenizer,  # 新しいAPI
)

print(f"✅ DPOTrainer初期化完了")
print(f"🔧 使用API: TRL 0.18.1")

# メモリ使用量の最終確認
if torch.cuda.is_available():
    allocated = torch.cuda.memory_allocated() / 1024**3
    reserved = torch.cuda.memory_reserved() / 1024**3
    print(f"💾 GPU メモリ (割り当て済み): {allocated:.2f} GB")
    print(f"💾 GPU メモリ (予約済み): {reserved:.2f} GB")

## 🏃‍♂️ 6. DPOトレーニング実行

**注意**: トレーニングには30分〜1時間程度かかる場合があります。

In [ ]:
# トレーニング開始時刻の記録
start_time = datetime.now()
print(f"🚀 DPOトレーニング開始: {start_time.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"📊 予想所要時間: 30-60分")
print(f"📈 進捗は下記で確認できます:")

try:
    # トレーニング実行
    trainer.train()
    
    # 完了時刻の計算
    end_time = datetime.now()
    duration = end_time - start_time
    
    print(f"\n✅ トレーニング完了!")
    print(f"⏱️  所要時間: {duration}")
    print(f"🎯 最終ステップ: {trainer.state.global_step}")
    
except Exception as e:
    print(f"❌ トレーニングエラー: {e}")
    # エラーログの詳細表示
    import traceback
    traceback.print_exc()

## 💾 7. モデル保存

In [ ]:
# 最終モデルの保存
final_output_dir = os.path.join(CONFIG["output_dir"], "final_model")
os.makedirs(final_output_dir, exist_ok=True)

# LoRAアダプターの保存
trainer.save_model(final_output_dir)
tokenizer.save_pretrained(final_output_dir)

print(f"💾 モデル保存完了: {final_output_dir}")

# 保存されたファイルの確認
saved_files = os.listdir(final_output_dir)
print(f"📁 保存されたファイル:")
for file in saved_files:
    print(f"  - {file}")

# Hugging Face Hubへのアップロード（オプション）
print(f"\n🚀 Hugging Face Hubにアップロードする場合:")
print(f"  trainer.push_to_hub('your-username/tiny-swallow-dpo-adtech')")

## 🧪 8. 学習済みモデルのテスト

In [ ]:
# テスト用プロンプト
test_prompts = [
    "プログラマティック広告のメリットを教えてください。",
    "DSPの主要な機能について説明してください。",
    "クッキーレス時代の対策について教えてください。",
    "広告のアトリビューション分析とは何ですか？",
    "ヘッダービディングの仕組みを説明してください。"
]

print("🧪 学習済みモデルのテスト開始")
print("=" * 80)

# テスト実行
for i, prompt in enumerate(test_prompts, 1):
    print(f"\n🔍 テスト {i}: {prompt}")
    print("-" * 60)
    
    try:
        # トークン化
        inputs = tokenizer.encode(prompt, return_tensors="pt").to(model.device)
        
        # テキスト生成
        with torch.no_grad():
            outputs = model.generate(
                inputs,
                max_new_tokens=200,
                temperature=0.7,
                do_sample=True,
                no_repeat_ngram_size=3,
                pad_token_id=tokenizer.eos_token_id,
            )
        
        # 生成テキストのデコード
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        response = generated_text[len(prompt):].strip()
        
        print(f"📝 回答: {response}")
        
    except Exception as e:
        print(f"❌ エラー: {e}")

print("\n" + "=" * 80)
print("🎉 テスト完了！")

## 📈 9. トレーニング結果の可視化

In [ ]:
# TensorBoardの起動
%load_ext tensorboard
%tensorboard --logdir={CONFIG["output_dir"]}/runs

print("📈 TensorBoard起動完了")
print("📊 トレーニングメトリクスを上記で確認できます")

## 📥 10. モデルのダウンロード（オプション）

In [ ]:
# 学習済みモデルをZIPファイルとしてダウンロード
import shutil
from google.colab import files

# ZIPファイルの作成
zip_filename = "tiny_swallow_dpo_model"
shutil.make_archive(zip_filename, 'zip', final_output_dir)

print(f"📦 ZIPファイル作成完了: {zip_filename}.zip")
print(f"📁 サイズ: {os.path.getsize(f'{zip_filename}.zip') / 1024**2:.1f} MB")

# ダウンロード
# files.download(f"{zip_filename}.zip")
print("💾 上記のコメントアウトを解除してダウンロードできます")

## 🎉 完了！

### ✅ 達成項目
- ✅ TinySwallow-1.5B モデルの読み込み
- ✅ 広告技術分野のDPOデータセット作成
- ✅ LoRA + 4bit量子化による効率的なファインチューニング
- ✅ TRL 0.18.1 最新APIを使用したDPOトレーニング
- ✅ 学習済みモデルのテスト
- ✅ トレーニング結果の可視化

### 📚 次のステップ
1. **実際のデータでの訓練**: より大規模で多様な広告技術データセットを使用
2. **ハイパーパラメータ調整**: 学習率、betaパラメータの最適化
3. **評価メトリクス**: より詳細な評価指標の導入
4. **プロダクション展開**: API化やWebアプリケーションとしての展開

### 🔗 参考リンク
- [TRL Documentation](https://huggingface.co/docs/trl/)
- [DPO Paper](https://arxiv.org/abs/2305.18290)
- [Transformers Documentation](https://huggingface.co/docs/transformers/)

---
**🦆 TinySwallow DPO Training Completed Successfully! 🎊**